## 데이타 전처리

In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [3]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
train_data_path = os.path.join(data_dir, 'merged_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,1951,직장 내 괴롭힘 대화,팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...
1,4756,일반 대화,"내일 날씨 어떻대?\n비 온다던데. 우산 챙겨가야 할 것 같아.\n에이, 야외 활동..."
2,1234,기타 괴롭힘 대화,야 쟤 좀 봐.\n 꼴에 유행하는 옷 입었네 \n 호박에 줄 긋는다고 수박되나 \n...
3,4767,일반 대화,오늘 수업 내용 이해했어?\n솔직히 좀 어려웠어. 너는?\n나도 몇 부분이 헷갈리더...
4,1511,직장 내 괴롭힘 대화,자네 내일 아침에 우리집 들려서 출근하게\n네?\n내가 차가 고장났어\n아. 그런데...


In [4]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 4637
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4637 entries, 0 to 4636
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           4637 non-null   int64 
 1   class         4637 non-null   object
 2   conversation  4637 non-null   object
dtypes: int64(1), object(2)
memory usage: 108.8+ KB


**한국어 불용어 사전**

In [32]:
#stopwords = ['나', '너', '네', '내','어', '고', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [47]:
with open('../../data/koreanStopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [18]:
import re

def clean_text(text):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)  # 한글, 영문, 공백 제외한 문자 제거
    text = text.replace('\n', ' ')
    text = text.strip()  # 양 끝 공백 제거
    return text

In [ ]:
#workpalce = [상사, 부서장, 팀장, 후배, 상급자, 부하직원, 선배, 동료, 부서, 팀, 부서원, 간호사, 간호부서, 간호단위 관리자]

In [ ]:
#bullying = [소리 지르다, 화내다, 업무 떠넘기다, 업무 제외, 업무 감시, 업무 간섭, 차별 대우, 비협조, 비꼬다, 모욕하다, 욕하다, 안 좋은 소문, 비방하다, 험담하다, 의심하다, 차가운 말투, 의심하는 눈빛, 차별, 불이익, 압박, 강요, 억압, 배제, 따돌림, 무시, 무관심 ]

In [26]:
df = train_data

**Mecab**

In [25]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

# Mecab : 형태소 분석기
#         형태소 추출(morphs), 명사 추출, 형태소와 태그 추출 
tokenizer = Mecab()

all_tokens = []
for sentence in df['conversation']:
    clean_sentence = clean_text(sentence)
    temp_X = tokenizer.morphs(clean_sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    all_tokens.append(temp_X)

words = np.concatenate(all_tokens).tolist()
#flattened_tokens = [token for sublist in all_tokens for token in sublist]
    
all_freqs = Counter(words).most_common(50)
print("전체 상위 단어")
print(all_freqs)


전체 상위 단어
[('하', 9853), ('어', 9167), ('고', 7436), ('야', 7180), ('아', 5893), ('거', 5489), ('있', 5208), ('지', 5161), ('해', 4813), ('안', 4028), ('게', 4003), ('다', 3958), ('을', 3795), ('말', 3658), ('면', 3509), ('겠', 3397), ('아니', 3214), ('없', 2967), ('주', 2661), ('뭐', 2646), ('왜', 2556), ('만', 2501), ('알', 2423), ('좋', 2386), ('니', 2348), ('습니다', 2151), ('돈', 1997), ('님', 1982), ('일', 1934), ('죄송', 1892)]


In [44]:
# 클래스별 빈도 계산
class_freqs = {}
for label in df['class'].unique():
    class_tokens = []
    sub_df = df[df['class'] == label]
    for conv in sub_df['conversation']:
        clean_sentence = clean_text(conv)
        tokens = tokenizer.morphs(clean_sentence) # 토큰화
        tokens = [word for word in tokens if not word in stopwords] # 불용어 제거
        class_tokens.extend(tokens)
    
    class_freqs[label] = Counter(class_tokens).most_common(50)

for key, value in class_freqs.items():
    print(f"\n [{key}] 상위 단어:")
    print(value)


 [직장 내 괴롭힘 대화] 상위 단어:
[('는', 2166), ('고', 1770), ('네', 1591), ('에', 1276), ('거', 1243), ('도', 1231), ('야', 1181), ('습니다', 1119), ('죄송', 1055), ('아', 1025), ('어', 1008), ('겠', 994), ('해', 992), ('합니다', 983), ('님', 981), ('은', 894), ('게', 775), ('면', 753), ('을', 743), ('다', 739), ('너', 563), ('제', 562), ('저', 554), ('뭐', 539), ('왜', 484), ('대리', 448), ('회사', 447), ('는데', 442), ('했', 441), ('할', 420), ('못', 416), ('시', 398), ('를', 381), ('만', 379), ('요', 365), ('오늘', 345), ('라고', 326), ('로', 315), ('김', 310), ('부장', 307), ('으로', 306), ('니', 303), ('어요', 289), ('무슨', 287), ('서', 283), ('그럼', 275), ('까지', 270), ('기', 253), ('세요', 252), ('너무', 238)]

 [일반 대화] 상위 단어:
[('어', 3192), ('에', 1482), ('는', 1406), ('아', 1264), ('은', 969), ('해', 935), ('고', 863), ('요즘', 821), ('도', 808), ('야', 691), ('을', 648), ('뭐', 624), ('다', 575), ('겠', 497), ('정말', 412), ('게', 403), ('어떻게', 395), ('오늘', 380), ('었', 375), ('이번', 353), ('주말', 351), ('네', 343), ('응', 338), ('먹', 336), ('재밌', 320), ('했', 318), ('영화'

**Okt**

In [19]:
from konlpy.tag import Okt

okt = Okt()

# 품사 태깅
def pos_tagging(text):
    # 관형사, 감탄사, 조사, 어미 등은 불용어 대상으로 토큰화 과정에 제외
    # 부사는 우리의 task에 중요한 정보라 판단하여 포함("진짜 짜증나", "너 정말 못됐다")
    return [word for word, pos in okt.pos(text) if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]

In [20]:
from collections import Counter


# 전체 품사별 빈도수 계산
all_tokens = []
for conv in df['conversation']:
    tokens = pos_tagging(clean_text(conv))
    tokens = [word for word in tokens if not word in stopwords] # 불용어 제거
    all_tokens.extend(tokens)

all_freqs = Counter(all_tokens).most_common(30)
print("전체 상위 단어")
print(all_freqs)

전체 상위 단어
[('너', 4622), ('내', 3896), ('나', 2720), ('말', 2640), ('네', 2599), ('왜', 2485), ('뭐', 2377), ('거', 2262), ('좀', 2073), ('돈', 1930), ('해', 1752), ('다', 1750), ('있어', 1583), ('것', 1513), ('이', 1481), ('진짜', 1470), ('죄송합니다', 1425), ('저', 1412), ('지금', 1407), ('니', 1402), ('아니', 1364), ('제', 1263), ('그럼', 1252), ('그래', 1215), ('오늘', 1157), ('우리', 1112), ('안', 1109), ('생각', 1094), ('할', 1049), ('요즘', 1043)]


In [45]:
# 협박 대화 빈도수 계산
subset = df[ (df['class'] == "협박 대화") ]
all_tokens = []

for conv in subset['conversation']:
    tokens = pos_tagging(clean_text(conv))
    tokens = [word for word in tokens if not word in stopwords] # 불용어 제거
    all_tokens.extend(tokens)

all_freqs = Counter(all_tokens).most_common(100)
print("협박 상위 단어")
print(all_freqs)

협박 상위 단어
[('너', 1506), ('왜', 599), ('니', 570), ('다', 492), ('제발', 461), ('해', 433), ('네', 409), ('거', 392), ('뭐', 342), ('진짜', 338), ('그래', 286), ('제', 252), ('그냥', 248), ('죽여', 248), ('죄송합니다', 240), ('저', 233), ('돈', 228), ('할', 225), ('그럼', 218), ('하면', 217), ('정말', 202), ('어떻게', 201), ('무슨', 199), ('새끼', 195), ('못', 185), ('당신', 172), ('봐', 171), ('가족', 170), ('있어', 165), ('난', 164), ('칼', 164), ('여기', 163), ('넌', 161), ('그렇게', 161), ('신고', 161), ('아니야', 161), ('당장', 157), ('이제', 146), ('살려주세요', 143), ('딸', 141), ('애', 137), ('어디', 133), ('죽', 130), ('빨리', 129), ('한번', 129), ('나도', 127), ('이렇게', 125), ('하지', 122), ('돼', 121), ('죽어', 120), ('전', 120), ('줄', 119), ('없어', 118), ('경찰', 117), ('하는', 116), ('그게', 116), ('뭘', 114), ('날', 113), ('오늘', 112), ('버릴거야', 112), ('게', 101), ('정신', 100), ('눈', 98), ('했어', 97), ('그건', 95), ('걸', 94), ('입', 93), ('미안해', 93), ('같아', 92), ('너무', 91), ('건', 90), ('그러면', 87), ('누구', 86), ('하겠습니다', 86), ('친구', 83), ('자꾸', 82), ('같이', 81), ('협박', 81), ('엄마

In [42]:
# 괴롭힘 대화 빈도수 계산
subset = df[ (df['class'] == "직장 내 괴롭힘 대화") | (df['class'] == "기타 괴롭힘 대화")]
all_tokens = []

for conv in subset['conversation']:
    tokens = pos_tagging(clean_text(conv))
    tokens = [word for word in tokens if not word in stopwords] # 불용어 제거
    all_tokens.extend(tokens)

all_freqs = Counter(all_tokens).most_common(100)
print("괴롭힘 상위 단어")
print(all_freqs)

괴롭힘 상위 단어
[('왜', 1327), ('말', 1285), ('죄송합니다', 1095), ('뭐', 1059), ('거', 984), ('아니', 782), ('해', 751), ('저', 750), ('제', 730), ('다', 712), ('진짜', 629), ('일', 628), ('지금', 618), ('것', 590), ('니', 547), ('우리', 542), ('그래', 538), ('안', 531), ('사람', 485), ('그럼', 479), ('그냥', 470), ('오늘', 462), ('봐', 457), ('대리', 457), ('회사', 456), ('그렇게', 447), ('무슨', 431), ('할', 403), ('너무', 400), ('하는', 364), ('알', 355), ('애', 344), ('생각', 337), ('부장', 317), ('이렇게', 311), ('여기', 307), ('더', 307), ('그', 298), ('못', 293), ('때', 286), ('새끼', 281), ('아니야', 280), ('김', 279), ('이번', 274), ('어떻게', 273), ('집', 270), ('시간', 270), ('그게', 263), ('하면', 261), ('하나', 256), ('말씀', 253), ('요', 246), ('넌', 245), ('아닙니다', 241), ('뭘', 239), ('자네', 237), ('빨리', 237), ('하겠습니다', 237), ('돈', 234), ('그건', 233), ('응', 231), ('하지', 229), ('과장', 228), ('하고', 221), ('팀', 219), ('해서', 219), ('수', 219), ('정말', 215), ('그런', 214), ('친구', 207), ('내일', 204), ('그래도', 204), ('걸', 204), ('다시', 203), ('근데', 202), ('앞', 201), ('업무', 191), ('

In [46]:
# 클래스별 빈도 계산
class_freqs = {}
for label in df['class'].unique():
    class_tokens = []
    sub_df = df[df['class'] == label]
    for conv in sub_df['conversation']:
        tokens = pos_tagging(clean_text(conv))
        tokens = [word for word in tokens if not word in stopwords] # 불용어 제거
        class_tokens.extend(tokens)
        
    class_freqs[label] = Counter(class_tokens).most_common(50)

for key, value in class_freqs.items():
    print(f"\n [{key}] 상위 단어:")
    print(value)


 [직장 내 괴롭힘 대화] 상위 단어:
[('네', 1095), ('죄송합니다', 871), ('너', 542), ('제', 528), ('저', 509), ('거', 483), ('뭐', 482), ('왜', 478), ('대리', 455), ('회사', 447), ('해', 418), ('다', 413), ('오늘', 344), ('부장', 316), ('그럼', 274), ('김', 272), ('할', 260), ('그래', 233), ('과장', 228), ('자네', 228), ('무슨', 221), ('아닙니다', 215), ('하겠습니다', 212), ('하는', 211), ('팀', 210), ('이번', 210), ('그렇게', 207), ('너무', 195), ('업무', 188), ('봐', 187), ('진짜', 185), ('말씀', 184), ('이렇게', 180), ('그냥', 175), ('하면', 173), ('내일', 170), ('휴가', 168), ('요', 164), ('못', 161), ('알겠습니다', 158), ('그게', 152), ('그건', 145), ('해서', 142), ('하고', 141), ('그래도', 135), ('커피', 135), ('어떻게', 131), ('빨리', 129), ('니', 129), ('정말', 129)]

 [일반 대화] 상위 단어:
[('있어', 1100), ('요즘', 821), ('뭐', 623), ('정말', 412), ('같아', 407), ('어떻게', 396), ('오늘', 380), ('좋은', 378), ('이번', 353), ('주말', 351), ('해', 340), ('응', 338), ('영화', 309), ('나도', 304), ('어때', 273), ('고마워', 272), ('계획', 271), ('좋아', 261), ('했어', 247), ('할', 242), ('됐어', 229), ('같이', 218), ('책', 218), ('어땠어', 200

**TF-IDF**

자주 등장하지만 모든 문서에 다 나오는 단어는 가중치를 낮게,
특정 문서에서만 자주 등장하는 단어는 가중치를 높게 줌

In [60]:
df['tokenized'] = df['conversation'].apply(lambda x: ' '.join(pos_tagging(clean_text(x))))

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['tokenized'])

# 단어 목록
print(vectorizer.get_feature_names_out())

# TF-IDF 행렬 확인
print(X_tfidf.toarray())

#print(X_tfidf)


['가가' '가감' '가거든요' ... '힘좀' '힙니' '힙합']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [62]:
import numpy as np

# 0번째 문장의 비어있지 않은 단어만 확인
row_0 = X_tfidf[0].toarray().flatten()
non_zero_indices = np.where(row_0 > 0)[0]
words = vectorizer.get_feature_names_out()

for i in non_zero_indices:
    print(f"{words[i]}: {row_0[i]:.4f}")

과장: 0.2091
넉넉하게: 0.1926
늦어지게: 0.2020
됐고: 0.1084
될까: 0.1183
됩니다: 0.1277
마무리: 0.3919
무리하지: 0.1926
받았어요: 0.1860
보내: 0.1199
생겼네: 0.1606
아뇨: 0.1247
아니: 0.0582
아직: 0.1823
알겠습니다: 0.0874
어떡합니까: 0.1766
언제: 0.0904
없더라구요: 0.2020
오늘: 0.1942
자리: 0.1114
작성: 0.1312
장님: 0.2257
전달: 0.2821
죄송해요: 0.1010
주말: 0.1725
퇴근: 0.1136
하라: 0.1014
하면: 0.1430
하셔서: 0.1353
해달라: 0.1860
해서: 0.0782
해주시면: 0.1477
했나요: 0.1446
했는데: 0.1874
했어요: 0.1121


In [63]:
classes = df['class'].unique()

for label in classes:
    subset = df[df['class'] == label]['conversation']
    tfidf = vectorizer.fit_transform(subset)
    feature_names = vectorizer.get_feature_names_out()
    
    mean_scores = tfidf.mean(axis=0).A1  # 평균 TF-IDF
    top_indices = mean_scores.argsort()[::-1][:50]
    
    print(f"\n📌 클래스: {label}")
    for i in top_indices:
        print(f"{feature_names[i]}: {mean_scores[i]:.4f}")


📌 클래스: 직장 내 괴롭힘 대화
죄송합니다: 0.0404
내가: 0.0254
아니: 0.0210
제가: 0.0197
지금: 0.0191
그럼: 0.0167
부장님: 0.0167
오늘: 0.0160
아닙니다: 0.0159
무슨: 0.0141
알겠습니다: 0.0133
그래: 0.0130
그렇게: 0.0128
너무: 0.0125
진짜: 0.0120
김대리: 0.0118
그냥: 0.0117
이거: 0.0112
회사: 0.0111
이렇게: 0.0111
과장님: 0.0109
그게: 0.0108
자네: 0.0106
다시: 0.0104
그건: 0.0101
일이: 0.0101
제대로: 0.0099
정말: 0.0098
우리: 0.0098
하고: 0.0098
그래도: 0.0097
어떻게: 0.0094
일을: 0.0092
하겠습니다: 0.0090
내일: 0.0088
빨리: 0.0088
다른: 0.0084
같습니다: 0.0083
요즘: 0.0082
아니요: 0.0081
저도: 0.0080
휴가: 0.0079
그런: 0.0078
같이: 0.0077
이건: 0.0075
팀장님: 0.0074
없어: 0.0073
사람이: 0.0072
근데: 0.0072
하는: 0.0070

📌 클래스: 일반 대화
있어: 0.0653
요즘: 0.0566
정말: 0.0294
어떻게: 0.0292
같아: 0.0289
오늘: 0.0281
좋은: 0.0276
주말에: 0.0246
영화: 0.0246
이번: 0.0243
어떤: 0.0239
나도: 0.0235
어때: 0.0230
좋아: 0.0225
고마워: 0.0216
같이: 0.0199
됐어: 0.0195
어땠어: 0.0185
그럼: 0.0183
싶어: 0.0183
좋아해: 0.0179
시간: 0.0179
그거: 0.0175
지내: 0.0164
재밌는: 0.0162
나는: 0.0157
거야: 0.0155
새로: 0.0153
계획: 0.0134
그렇게: 0.0134
생각이야: 0.0133
봤어: 0.0133
커피: 0.0133
그래서: 0.0131
저녁: 0.01

In [49]:
# 특정 단어에 가중치 추가
custom_vocab = {'협박': 3, '죽인다': 5, '조심해': 2}

def custom_tokenizer(text):
    return text.split()

vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, vocabulary=custom_vocab)
X_custom = vectorizer.fit_transform(df['conversation'])

print(X_custom.toarray())

ValueError: Vocabulary of size 3 doesn't contain index 0.

**원-핫 인코딩(One-Hot Encoding)**

In [6]:
# class 컬럼 원-핫 인코딩
df = train_data
class_dummies = pd.get_dummies(df['class'])
print(class_dummies)

      갈취 대화  기타 괴롭힘 대화  일반 대화  직장 내 괴롭힘 대화  협박 대화
0         0          0      0            1      0
1         0          0      1            0      0
2         0          1      0            0      0
3         0          0      1            0      0
4         0          0      0            1      0
...     ...        ...    ...          ...    ...
4632      0          0      0            1      0
4633      0          0      0            0      1
4634      1          0      0            0      0
4635      1          0      0            0      0
4636      0          0      1            0      0

[4637 rows x 5 columns]


In [8]:
class_order = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']
class_dummies = class_dummies[class_order]
print(class_dummies)

      협박 대화  갈취 대화  직장 내 괴롭힘 대화  기타 괴롭힘 대화  일반 대화
0         0      0            1          0      0
1         0      0            0          0      1
2         0      0            0          1      0
3         0      0            0          0      1
4         0      0            1          0      0
...     ...    ...          ...        ...    ...
4632      0      0            1          0      0
4633      1      0            0          0      0
4634      0      1            0          0      0
4635      0      1            0          0      0
4636      0      0            0          0      1

[4637 rows x 5 columns]


In [16]:
train_labels = class_dummies.to_numpy()
train_labels

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [13]:
# 기존 df에 인코딩 결과 붙이기
df_with_dummies = pd.concat([df, class_dummies], axis=1)

print(df_with_dummies[:3])

    idx        class                                       conversation  \
0  1951  직장 내 괴롭힘 대화  팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...   
1  4756        일반 대화  내일 날씨 어떻대?\n비 온다던데. 우산 챙겨가야 할 것 같아.\n에이, 야외 활동...   
2  1234    기타 괴롭힘 대화  야 쟤 좀 봐.\n 꼴에 유행하는 옷 입었네 \n 호박에 줄 긋는다고 수박되나 \n...   

   협박 대화  갈취 대화  직장 내 괴롭힘 대화  기타 괴롭힘 대화  일반 대화  
0      0      0            1          0      0  
1      0      0            0          0      1  
2      0      0            0          1      0  


In [ ]:
#beomi/kcbert-base는 욕설, 위협, 혐오 발언 등 비일상적인 표현이 많은 데이터셋에 특히 적합합니다.
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
#tokens = tokenizer("너 정말 죽고 싶냐?", return_tensors='pt')

In [ ]:
#형대소 분석기 기반 토크나이저 
# Mecab